In [38]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#!pip install folium
import folium
import json
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_colwidth', 1000)
pd.set_option('max_rows', 30)
pd.set_option('max_columns', 1000)

In [2]:
#Testing API connection
import urllib.request as req
url = 'https://datos.cdmx.gob.mx/api/3/action/datastore_search?resource_id=d543a7b1-f8cb-439f-8a5c-e56c5479eeb5&limit=1000000'  
fileobj = req.urlopen(url)
print (fileobj.read)

<bound method HTTPResponse.read of <http.client.HTTPResponse object at 0x000001DF43FD27F0>>


In [3]:
type(fileobj)

http.client.HTTPResponse

In [5]:
response = json.loads(fileobj.read())

In [32]:
result = response.get('result')
records = result.get('records')

In [39]:
df = pd.DataFrame(records)
df.head(2)

,_id,idCarpeta,Ano_inicio,Mes_inicio,FechaInicio,Delito,Categoria,Sexo,Edad,TipoPersona,CalidadJuridica,competencia,Ano_hecho,Mes_hecho,FechaHecho,HoraHecho,HoraInicio,AlcaldiaHechos,ColoniaHechos,Calle_hechos,Calle_hechos2,latitud,longitud
0,63881,8348939,2019,Febrero,2019-05-02T00:00:00,FRAUDE,DELITO DE BAJO IMPACTO,Masculino,NA,FISICA,VICTIMA Y DENUNCIANTE,FUERO COMUN,2019,Enero,20/01/2019,09:00:00,12:42:56,GUSTAVO A MADERO,ROSAS DEL TEPEYAC,ALLENDE,NA,19.48936,-99.11665
1,63882,8348941,2019,Febrero,2019-05-02T00:00:00,ROBO DE OBJETOS A ESCUELA,DELITO DE BAJO IMPACTO,NA,NA,MORAL,OFENDIDO,FUERO COMUN,2019,Febrero,02/02/2019,11:00:00,12:44:34,BENITO JUAREZ,PORTALES SUR,EMPERADORES,AVENIDA TLALPAN,19.36554,-99.14266


In [37]:
print(df.columns)
print(df.info())

Index(['_id', 'idCarpeta', 'Ano_inicio', 'Mes_inicio', 'FechaInicio', 'Delito',
       'Categoria', 'Sexo', 'Edad', 'TipoPersona', 'CalidadJuridica',
       'competencia', 'Ano_hecho', 'Mes_hecho', 'FechaHecho', 'HoraHecho',
       'HoraInicio', 'AlcaldiaHechos', 'ColoniaHechos', 'Calle_hechos',
       'Calle_hechos2', 'latitud', 'longitud'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32000 entries, 0 to 31999
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   _id              32000 non-null  int64 
 1   idCarpeta        32000 non-null  int64 
 2   Ano_inicio       32000 non-null  int64 
 3   Mes_inicio       32000 non-null  object
 4   FechaInicio      32000 non-null  object
 5   Delito           32000 non-null  object
 6   Categoria        32000 non-null  object
 7   Sexo             32000 non-null  object
 8   Edad             32000 non-null  object
 9   TipoPersona      32000 n

In [41]:
df.Mes_inicio.value_counts()

Octubre       5655
Julio         4298
Abril         4006
Enero         3999
Diciembre     3774
Marzo         3723
Febrero       2682
Septiembre    2507
Junio         1308
Mayo            47
Noviembre        1
Name: Mes_inicio, dtype: int64

In [42]:
df.Categoria.value_counts()

DELITO DE BAJO IMPACTO                                     24181
ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA        1992
ROBO A NEGOCIO CON VIOLENCIA                                1718
ROBO DE VEHÍCULO CON Y SIN VIOLENCIA                        1686
ROBO A REPARTIDOR CON Y SIN VIOLENCIA                        499
HECHO NO DELICTIVO                                           459
ROBO A PASAJERO A BORDO DEL METRO CON Y SIN VIOLENCIA        378
HOMICIDIO DOLOSO                                             234
LESIONES DOLOSAS POR DISPARO DE ARMA DE FUEGO                233
VIOLACIÓN                                                    203
ROBO A PASAJERO A BORDO DE MICROBUS CON Y SIN VIOLENCIA      152
ROBO A CASA HABITACIÓN CON VIOLENCIA                          91
ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON VIOLENCIA       79
ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA                 59
ROBO A TRANSPORTISTA CON Y SIN VIOLENCIA                      25
SECUESTRO                